In [1]:
import geemap
import ee
from Monarch import user_gee

In [2]:
Map = geemap.Map(center=(30, 112))
Map

Map(center=[30, 112], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [3]:
roi = geemap.shp_to_ee(r'H:/Monarch/paper/Data/jiujiang.shp').geometry()
Map.center_object(roi, 10)
Map.addLayer(roi, {}, 'roi')

In [ ]:
roi = Map.draw_last_feature.geometry()

In [4]:
visParams = {'max': 3000, 'min': 0, 'bands': ['B4', 'B3', 'B2']}

In [8]:
for year in range(2016, 2020):
    s2_year = []
    for i in range(3, 12):
        s2 = ee.ImageCollection("COPERNICUS/S2")\
                .filterBounds(roi)\
                .sort('CLOUD_COVERAGE_ASSESSMENT', False)\
                .filterDate(str(year), str(year+1))\
                .filter(ee.Filter.calendarRange(i, i, 'month'))
        s2_z = s2.merge(s2.map(user_gee.rm_cloud_s2_sr))
        a = s2.aggregate_array('system:time_start').map(lambda i: ee.Date(i).format('YYYY-MM-dd'))
        s2_year.append(s2_z.mosaic().clip(roi).set('time', a.get(s2.size().subtract(1))))
    s2_year = ee.ImageCollection(s2_year)
    geemap.download_ee_video(s2_year, visParams, f'H:/Monarch/paper/Data/S2_image_Thumbnail_git/S2_{year}.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: H:\Monarch\paper\Data\S2_image_Thumbnail_git\S2_2016.gif
Generating URL...
Please wait ...
The GIF image has been saved to: H:\Monarch\paper\Data\S2_image_Thumbnail_git\S2_2017.gif
Generating URL...
Please wait ...
The GIF image has been saved to: H:\Monarch\paper\Data\S2_image_Thumbnail_git\S2_2018.gif
Generating URL...
Please wait ...
The GIF image has been saved to: H:\Monarch\paper\Data\S2_image_Thumbnail_git\S2_2019.gif


In [ ]:
geemap.add_text_to_gif(f'H:/Monarch/paper/Data/S2_image_Thumbnail_git/S2_{year}.gif', f'H:/Monarch/paper/Data/S2_image_Thumbnail_git/S2_{year}_3.gif', xy=('3%', '5%'), text_sequence=a, 
                       font_size=20, font_color='#ffffff', duration=1000)

In [ ]:
a = s2_year.aggregate_array('time').getInfo()

In [ ]:
a